# Imports

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt


# Extract PEN

In [ ]:
# load and resize img
img = cv2.imread('./source_code/exam_1.jpg', cv2.IMREAD_UNCHANGED)
# img = cv2.imread('/home/dom/git/paper/source_code/exam_1.jpg', 1)
width = 500
height = 500
dim = (width, height)
  
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

In [ ]:
BLUR_RADIUS = 5

img = cv2.blur(img, (BLUR_RADIUS,BLUR_RADIUS))
img = cv2.medianBlur(img, 11)

cv2.imwrite('output/extract_pen/img_filter.jpg', img)
#cv2.imshow('filter img', img)
#cv2.waitKey(0)

rows, cols = img.shape[:2]
for x in range(0, rows -1):
  for y in range(0, cols -1):

    if img[x, y][0] > 200 and img[x, y][1] > 200 and img [x, y][2] > 200:
      img[x, y][0] = 255
      img[x, y][1] = 255
      img[x, y][2] = 255


# OPERAÇÃO 1
#EROSION_TYPE = 'MORPH_RECT' # MORPH_ELLIPSE; MORPH_RECT; MORPH_CROSS
erosion_type = 'MORPH_RECT' # MORPH_ELLIPSE; MORPH_RECT; MORPH_CROSS
erosion_size = 1 
element = cv2.getStructuringElement(
    cv2.MORPH_RECT  ,
    (2 * erosion_size + 1, 2 * erosion_size + 1),
    (erosion_size, erosion_size)
)

img = cv2.dilate(img, element)
img = cv2.dilate(img, element)

cv2.imwrite('output/extract_pen/img_operacao_1.jpg', img)

# OPERAÇÃO 2
erosion_size = 2

element1 = cv2.getStructuringElement(
    cv2.MORPH_RECT  ,
    (2 * erosion_size + 1, 2 * erosion_size + 1),
    (erosion_size, erosion_size)
)

img = cv2.erode(img, element1)
 
for y in range(0, rows -1):

  for x in range(0, cols -1 ):
    # TODO: DESCOBRIR O QUE SIGNIFICA ESSA OPERAÇAO
    difRG = abs(img[x,y][0] - img[x,y][1])
    difRB = abs(img[x,y][0] - img[x,y][2])
    difGB = abs(img[x,y][1] - img[x,y][2])

    if(difGB < 30 and difRB < 30 and difRG < 40):
      img[x,y][0] = 255
      img[x,y][1] = 255
      img[x,y][2] = 255

img = cv2.blur(img, (BLUR_RADIUS, BLUR_RADIUS))
img = cv2.medianBlur(img, 3)

cv2.imwrite('output/extract_pen/img_operacao_2.jpg', img)

# Extract Template

In [ ]:
# load and resize img
img = cv2.imread('./source_code/exam_1.jpg', cv2.IMREAD_UNCHANGED)
# img = cv2.imread('/home/dom/git/paper/source_code/exam_1.jpg', 1)
width = 500
height = 500
dim = (width, height)
  
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

In [ ]:
BLUR_RADIUS = 5
img = cv2.blur(img,(BLUR_RADIUS, BLUR_RADIUS))

rows, cols = img.shape[:2]

for y in range(rows - 1):
  for x in range(cols - 1):
    color = img.copy()
    if color[x, y][0] < 120 and color[x, y][1] < 120 and color[x, y][2] < 120:
      img[x, y][0] = 0
      img[x, y][1] = 0
      img[x, y][2] = 0
    
    else:
      img[x, y][0] = 255
      img[x, y][1] = 255
      img[x, y][2] = 255


# OPERACÃO 1
erosion_type = 'MORPH_RECT' # MORPH_ELLIPSE; MORPH_CROSS
erosion_size = 2

element = cv2.getStructuringElement(
    cv2.MORPH_RECT, (2 * erosion_size + 1, 2 * erosion_size + 1),
    ( erosion_size, erosion_size)
)

img = cv2.erode(img, element)

cv2.imwrite('output/extract_template/template_1.jpg', img)

# OPERAÇÃO  2
# Apply the dilate and erosin operations to remove noise and debris in the scanned image
erosion_size = 1

element1 = cv2.getStructuringElement(
    cv2.MORPH_RECT, (2 * erosion_size + 1, 2 * erosion_size + 1),
    ( erosion_size, erosion_size)
)

img = cv2.dilate(img, element1)
img = cv2.dilate(img, element1)

img = cv2.blur(img, (3 * BLUR_RADIUS, 3 * BLUR_RADIUS))

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imwrite('output/extract_template/template_2.jpg', img)

element2 = cv2.getStructuringElement(
    cv2.MORPH_RECT,
    (2 * erosion_size + 1, 2 * erosion_size + 1),
    (erosion_size, erosion_size)
)
# OTSU THRESHOLD IS USE TO REMOVE THE NOISE AND BACKGROUND
# PARAMETRO 1 : IMAGEM EM CINZA
# PARAMETRO 2 : VALOR LIMIAR, RESOLVIDO TENTATIVA E ERRO
# PARAMETRO 3 : INTENSIADADE QUE OS PIXELS VAO RECEBER NOS VALORES SUPERIOR AO LIMIAR
# PARAMETRO 3 : METODO QUE SERÁ APLICADO
ret, img_thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  
img_thresh = cv2.dilate(img_thresh, element2)
img_thresh = cv2.dilate(img_thresh, element2)
img_thresh = cv2.erode(img_thresh, element2)
img_thresh = cv2.erode(img_thresh, element2)
img_thresh = cv2.erode(img_thresh, element2)

cv2.imwrite('output/extract_template/template_3.jpg', img_thresh)



# Extract Feats

In [3]:
# rotation
def rotation(vert, yp, xp, teta):
    xe = (
        np.float128(
            ((vert[x] - xp) * (math.cos(teta * math.pi / 180)))
            - ((vert[y] - yp) * (math.sin(teta * math.pi / 180)))
        )
        + xp
    )
    ye = (
        np.float128(
            ((vert[x] - xp) * (math.sin(teta * math.pi / 180)))
            + ((vert[y] - yp) * (math.cos(teta * math.pi / 180)))
        )
        + yp
    )

    vert = {x: xe, y: ye}


# Verify the central pixel
def verify(img_, y, x):
    cont = 0
    for i in range(y - 1, y + 2):
        for j in range(x - 1, x + 2):
            if img_[i, j] == 0:
                cont += 1

    return cont == 2


# find the spiral origem
def origem(img_, oy, ox):
    lines = int(oy + 100)
    cols = int(ox + 100)
    i, j = 0, 0
    com_line = int(oy - 100)
    com_col = int(ox - 100)

    for i in range(com_line, lines):
        for j in range(com_col, cols):
            if img_[i, j] == 0:
                if verify(img_, i, j):
                    ox = j
                    oy = i


def invert(start, end):
    tmp = start
    start = end
    end = tmp


def lineIDDA(img_, yi, xi, yf, xf, vetor_vertices):
    quant, erro, q = 0, 0, 0
    deltax = xf - xi
    deltay = xf - yi
    vert = {"x": 0, "y": 0}

    # When yi>yf the line is diagonal to down. For this is
    # necessary invert the points
    # When Deltax= 0 and Deltay  < 0, the line is in vertical
    # to down. For this is necessary invert yi with yf.

    if (yi > yf) or ((deltax == 0) and (deltay == 0)):
        invert(xi, xf)
        invert(yi, yf)

        deltax = int(xf - xi)
        deltax = int(xf - yi)

    x = xi
    y = yi

    # 'quant' denotes the maximum number of ploted points
    if abs(deltax) > abs(deltay):
        quant = abs(deltax)
    else:
        quant = abs(deltay)

    get_point = True
    entered = False
    finished = False
    walk = 1000  # clean more pixels

    while (q <= quant) and walk:

        if x >= 0 and y >= 0 and x < int(img_.shape[1]) and y < int(img_.shape[0]):

            if not entered and img_[y, x] == 0:  # find the spiral

                entered = True
                vert[x] = x
                vert[y] = y
                vetor_vertices.append(vert)

            img[y, x] = 255  # Set the color white to avoid reprocessing

        if entered:
            walk -= 1

            if x >= 0 and y >= 0 and x < int(img_.shape[1]) and y > int(img_.shape[0]):
                img[y, x] = 255  # Set the color white to avoid reprocessing

        if (deltax >= 0) and (deltay >= 0) and (deltax >= deltay):  # 1 oct

            if (erro < 0) or (deltay == 0):
                x += 1
                erro = erro + deltay
            else:
                x += 1
                y += 1
                erro = erro + deltay - deltax

        elif (deltax >= 0) and (deltay >= 0) and (deltay > deltax):  # 2 oct

            if erro < 0:
                x += 1
                y += 1
                erro = erro + deltay - deltax
            else:
                y += 1
                erro = erro - deltax

        elif (deltay >= 0) and (deltax < 0) and (-deltax >= deltay):  # 4 oct

            if (erro < 0) or deltay == 0:
                x -= 1
                erro = erro + deltay
            else:
                x -= 1
                y += 1
                erro = erro + deltax + deltay

        elif (deltay > 0) and (deltax < 0) and (deltay > -deltax):  # 3 oct

            if erro < 0:
                x -= 1
                y += 1
                erro = erro + deltax + deltay
            else:
                y += 1
                erro = erro + deltax

        elif (deltax >= 0) and (deltay < 0) and (deltax >= -deltay):  # 8 oct

            if erro < 0:
                x += 1
                erro = erro - deltay
            else:

                x += 1
                y -= 1
                erro = erro + abs(deltay) - deltax

        elif (deltax >= 0) and (deltay < 0) and (-deltay > deltax):  # 4 oct

            if erro < 0:
                x += 1
                y -= 1
                erro = erro - deltay - deltax
            else:
                y -= 1
                erro = erro - deltax

        elif (deltay < 0) and (deltax < 0) and (-deltay > -deltax):  # 3 oct

            if erro < 0:
                x -= 1
                y -= 1
                erro = erro + deltax - deltay
            else:
                y -= 1
                erro = erro + deltax
        elif (deltay < 0) and (deltax < 0) and (-deltax >= -deltay):  # 4 oct

            if erro < 0:
                x -= 1
                erro = erro - deltay
            else:
                x -= 1
                y -= 1
                erro = erro + deltax - deltay

        q += 1  # number of plotted points

In [4]:

# load and resize img
img = cv2.imread('./source_code/exam_1.jpg', cv2.IMREAD_COLOR)
img1 = cv2.imread('output/extract_pen/img_operacao_2.jpg',cv2.IMREAD_COLOR)
# img = cv2.imread('/home/dom/git/paper/source_code/exam_1.jpg', 1)
width = 500
height = 500
dim = (width, height)
  
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


In [16]:
DISPLACEMENT = 10

# lista de dicionarios com raio e angulo
radius_angle = []
radius_rigin = []
difradial = []

# lista de dicionarios com vertices x, y
ptos_original = []
ptos_desenhada = []
tremor_relativo = []

# lista de np.float(128)
tremor = []

# dicionario com valores de raios e anglus
ra = {"radius": 0, 'angle': 0}
ra_origin = {"radius": 0, 'angle': 0}

# recebe np.float128()
raiz, atangente = 0, 0

vert = {"x": 0, "y": 0}

# gray scale
#img_ = np.zeros((cols, rows), dtype='uint8')
img_gray = img.copy()
img_gray = cv2.cvtColor(img_gray, cv2.COLOR_BGR2GRAY)
ret, img_gray = cv2.threshold(img_gray, 220, 255, 0)
cv2.ximgproc.thinning(img_gray, cv2.ximgproc.THINNING_ZHANGSUEN)
# cv2.ximgproc.thinning(img_gray, cv2.ximgproc.THINNING_GUOHALL)

# cv2.imwrite('output/extract_feats/extract.jpg', img_gray)


# Spiral pen
img1_= img1.copy()
rows, cols = img1.shape[:2]

img1_gray = cv2.cvtColor(img1_, cv2.COLOR_BGR2GRAY)
#cv2.imwrite('output/extract_feats/extract_pen_gray.jpg', img1_gray)
ret, img1_gray = cv2.threshold(img1_gray, 220, 255, 0)
# cv2.imwrite('output/extract_feats/extract_pen_thresh.jpg', img1_gray)
img1_gray = cv2.ximgproc.thinning(img1_gray, cv2.ximgproc.THINNING_ZHANGSUEN)
# cv2.imwrite('output/extract_feats/extract_pen_thinning.jpg', img1_gray)

yc = int(img_gray.shape[0]/2)
xc = int(img_gray.shape[1]/2)

cv2.imwrite('output/tmp/1.jpg', img_gray)
origem(img1_gray, yc, xc)
cv2.imwrite('output/tmp/2.jpg', img_gray)

# Get point from spiral
for i in range(3): # 3 turns in the spiral
    vert['y'] = yc
    vert['x'] = cols + 350
    
    rotation(vert, yc, xc, 1)

    for i in range(360):
        pt_ori = int(len(ptos_original))
        pt_des = int(len(ptos_desenhada))

        rotation(vert, yc, xc, -1)
        lineIDDA(img1_gray, yc, xc, vert[y], vert[x], ptos_original)
        lineIDDA(img_gray, yc, xc, vert[y], vert[x], ptos_original)

#------------------------------------------------------------------------------ 
yc = int(img_gray.shape[0]/2)
xc = int(img_gray.shape[1]/2)


#------------------------------------------------------------------------------ 
# Transformation to polar coordinates
for i in range(0, len(ptos_original)):
    raiz = (ptos_original[i][x-xc]) * (ptos_original[i][x-xc]) + (ptos_original[i][y-yc]) * (ptos_original[i][y-yc])
    ra_origin["radius"] = math.sqrt(raiz) # computing the radius
    atangente = ptos_original[i][y-yc]/ptos_original[i][x-xc]
    ra_origin['angle']= math.atan(atangente) # computating the angel (Radianos)
    radius_rigin.append(ra_origin)


# Transformation to polar coordinates
for i in range(0, len(ptos_desenhada)):
    raiz = (ptos_desenhada[i][x-xc]) * (ptos_desenhada[i][x-xc]) + (ptos_desenhada[i][y-yc]) * (ptos_desenhada[i][y-yc])
    ra["radius"] = math.sqrt(raiz) # computing the radius
    atangente = ptos_desenhada[i][y-yc]/ptos_desenhada[i][x-xc]
    ra['angle']= math.atan(atangente) # computating the angel (Radianos)
    radius_rigin.append(ra)

#------------------------------------------------------------------------------ 
# Calculate the difference between the template and drawed spiral
dif_rad = 0.0
prev_rad = radius_rigin[0]["radius"] - radius_angle[0]["radius"]
count_cross = 0

ra["radius"] = abs(prev_rad)
difradial.append(ra)

iterator = 1
while iterator < len(ptos_desenhada) and iterator < len(ptos_desenhada):
    dif_rad = radius_rigin[iterator]['radius'] - radius_angle[i]['radius']
    ra['radius'] = abs(dif_rad)
    difradial.pb(ra)

    if ((dif_rad * prev_rad) < 0):
        count_cross += 1 # CROSS THE TEMPLATE 
    
    prev_rad = dif_rad
    iterator += 1
  


#------------------------------------------------------------------------------ 
# computating the Relative Tremor
iterator = 1
while iterator < len(ptos_desenhada) and iterator < len(ptos_original):
    raiz = ptos_original[iterator][x-xc] ** 2 + ptos_original[iterator][y-yc] ** 2
    vert['x'] = 1 - (difradial[i]['radius'] / math.sqrt(raiz)) * ptos_desenhada[i]['x']
    print(vert['x'])
    vert['y'] = 1 - (difradial[i]['radius'] / math.sqrt(raiz)) * ptos_desenhada[i]['y']
    print(vert['y']) 

    tremor_relativo.append(vert)
    iterator += 1

mean_tremor = 0.0
std_tremor = 0.0
max_tremor = 0.0
min_tremor = 10000.0
count = 0

iterator = 0
while iterator < len(ptos_desenhada) and  iterator < len(ptos_original):
    raiz = math.sqrt(ptos_original[i][x-xc] ** 2 + ptos_original[i][y-yc] ** 2)
    tremor.append(raiz)

for i in range(DISPLACEMENT, len(tremor)):
    dif = math.fabs(tremor[i] - tremor[i - DISPLACEMENT])
    mean_tremor += dif
    if dif > max_tremor: max_tremor= dif
    if dif < min_tremor: min_tremor = dif

mean_tremor /= np.float32

for i in range(DISPLACEMENT,len(tremor)):
    dif = math.fabs(tremor[i] - tremor[i - DISPLACEMENT])
    std_tremor += np.float32(math.pow(dif - mean_tremor, 2) / np.float32(len(tremor) - DISPLACEMENT))



#---------------------- Extracting features from tremor-------------------------
# Computating the RMS (Root Mean Square)

RMS = 0.0
minRMS = 10 ** 10
maxRMS = 0.0
std = 0.0
min_p = len(ptos_desenhada)

if min_p > len(ptos_desenhada):
    min_p = len(ptos_original)

iterator = 0
while iterator < len(ptos_desenhada) and iterator < len(ptos_original):
    RMS += difradial[iterator]['radius'] ** 2

    if (difradial[iterator]['radius'] ** 2 > maxRMS) :
        maxRMS = difradial[iterator]['radius'] ** 2
    
    if (difradial[iterator]['radius'] ** 2 < minRMS) :
        minRMS = difradial[iterator]['radius'] ** 2
    
    iterator += 1

RMS /= np.float32(min_p)

iterator = 0
while iterator < len(ptos_desenhada) and iterator < len(ptos_original):
    std += np.float32(math.pow(difradial[iterator]['radius'], 2) / np.float32(len(ptos_original)))

std = math.sqrt(std)

print(
    f'[{arg1} : {arg2}]\n'
    f'RMS: {RMS} (+/- {std} :\n'
    f'maxRMS: {maxRMS}' 
    f'minRMS: {minRMS}:\n'
    f'Npoints: {len(ptos_desenhada)}: {len(ptos_original)}\n'
    f'MT: {mean_tremor}'
    f'Max: {max_tremor}'
    f'Min: {min_tremor}'
    f'Std: {std_tremor}\n'
    f'Cross: {count_cross / count}'
    )

if arg == 'c':
    class_img = 1
elif arg == 'p':
    class_img = 2
else: 
    f'\n ERROR: unknown class. The class identifier needs to be "c" (control) or "p" (patient)\n'


output = f'{class_img}'
output += f'1: {RMS}'
output += f'2: {std}'
output += f'3: {maxRMS}'
output += f'4: {minRMS}'
output += f'5: {mean_tremor}'
output += f'6: {max_tremor}'
output += f'7: {min_tremor}'
output += f'8: {std_tremor}'
output += f'9: {count_cross / count}'
with open('RMS.txt','w', encoding= 'utf-8') as f:
    f.write(output)


KeyError: 498

In [ ]:
with open("test.txt",'w',encoding = 'utf-8') as f:
   f.write("my first file\n")
   f.write("This file\n\n")
   f.write("contains three lines\n")